In [ ]:
# !pip install langchain
# !pip install -U langchain-openai
# !pip install -U langchain_neo4j
# !pip install langchain_community
# !pip install neo4j
# !pip install shapely
# !pip install -U pyproj
# !pip install tqdm
# !pip install langchain-community
# !pip install -U pandas

# !pip install -U langchain-neo4j

In [2]:
# import json
# from tqdm import tqdm
from dotenv import load_dotenv
load_dotenv()

import os

from langchain_neo4j import Neo4jGraph
from langchain.chat_models import init_chat_model
from langgraph.graph import START, StateGraph
from langchain_core.prompts import PromptTemplate
from typing_extensions import List, TypedDict


import psycopg2
from neo4j import GraphDatabase
from decimal import Decimal
from shapely import wkb
from pyproj import CRS, Transformer


In [3]:

load_dotenv()

keys = ["NEO4J_USER", "NEO4J_PASSWORD","NEO4J_USER_LOCAL", "NEO4J_PASSWORD_LOCAL"]
for key in keys:
    if key not in os.environ:
        raise Exception(f"Key '{key}' not found not in .env")
    
print("Credentials from .env file successfully loaded")

NEO4J_LOCAL_DATABASE = "busoppdse203"

Credentials from .env file successfully loaded


In [6]:
graph = Neo4jGraph(url="bolt://localhost:7687",    username='neo4j',password='admin123',database=NEO4J_LOCAL_DATABASE )
graph.query("MATCH (n) RETURN n LIMIT 1;")


[{'n': {'name': 'Midtown', 'id': 60}}]

In [7]:
# Initialize the LLM
model_ver = "gpt-4o"
model_provider = 'openai'
model = init_chat_model(model_ver, model_provider=model_provider)


In [8]:
# Define state for application
class State(TypedDict):
    question: str
    context: List[dict]
    answer: str


In [9]:
# Retrieve context 
# def retrieve(state: State):
#     context = graph.query("CALL db.schema.visualization()")
#     return {"context": context}




def retrieve(state: State):
    schema_map = graph.query("CALL apoc.meta.schema()")
    
    context = schema_map 
    return {"context": context}

In [10]:
# Create a prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)



In [11]:
# Generate the answer based on the question and context
def generate(state: State):
    messages = prompt.invoke({"question": state["question"], "context": state["context"]})
    response = model.invoke(messages)
    return {"answer": response.content}







In [12]:
# Define application steps
workflow = StateGraph(State).add_sequence([retrieve, generate])
workflow.add_edge(START, "retrieve")
app = workflow.compile()


In [15]:
# entity_type = 'BlockGroup'
# question = "How is the graph structured?"



question = "What questions can you answer"
# question = f"What attributes are available on a {entity_type} node"
# question = "Are any cities contained in cities"

# question = 'What is the context'

response = app.invoke({"question": question})
print("Answer:", response["answer"])

Answer: Based on the context provided, I can answer the following types of questions:

1. **Node Count Questions**:
   - How many nodes exist for each type (e.g., Business, BlockGroup, City, etc.)?

2. **Relationship Count Questions**:
   - How many specific relationships exist (e.g., how many "adjacent_to" or "contained_in" relationships are there)?

3. **Node Property Questions**:
   - What properties are available for each node type (e.g., properties of Business, BlockGroup, etc.) and their data types?

4. **Relationships Details**:
   - What are the types and directions of relationships between node types (e.g., which nodes have "contained_in" or "adjacent_to" relationships)?

5. **Node Uniqueness and Indexing**:
   - Are properties of nodes unique or indexed?

If you have questions within these areas, I can provide answers based on the context given.


# Text 2 Cypher

In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langchain_neo4j import Neo4jGraph

# Initialize the LLM
model = init_chat_model(
    "gpt-4o", 
    model_provider="openai"
)


cypher_model = init_chat_model(
    "gpt-4o", 
    model_provider="openai",
    temperature=0.0
)


# Connect to Neo4j
graph = Neo4jGraph(
    url=os.getenv("NEO4J_URI_LOCAL"),
    username=os.getenv("NEO4J_USER_LOCAL"), 
    password=os.getenv("NEO4J_PASSWORD_LOCAL"),
)

# Create the Cypher QA chain

# Invoke the chain

In [26]:
from langchain_neo4j import GraphCypherQAChain

# # Create the Cypher QA chain
# cypher_qa = GraphCypherQAChain.from_llm(
#     graph=graph, 
#     llm=model, 
#     allow_dangerous_requests=True,
#     verbose=True, 
# )

cypher_qa = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=model, 
    cypher_llm=cypher_model,
    allow_dangerous_requests=True,
    verbose=True,
)


In [27]:

# Invoke the chain
question = "How many communities are there in San Diego"

response = cypher_qa.invoke({"query": question})
print(response["result"])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (c:Community)-[:contained_in]->(ci:City {name: "San Diego"})
RETURN count(c) AS numberOfCommunities

Full Context:
[{'numberOfCommunities': 157}]

> Finished chain.
There are 157 communities in San Diego.
